<a href="https://colab.research.google.com/github/hkangah3/SE-assignment2/blob/main/Group4_11_4_MemoryAllocationStrategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from numpy import random
import matplotlib.pyplot as plt

# memory size
TOTAL_UNITS = 100

# 8 bytes for every unit
UNIT_SIZE = 8

# negative indicates free units in the memory;
memory: list = [-TOTAL_UNITS]

d = 10  # default mean (center of the curve)
v = 1 # standard deviation (how wide the curve is)

def initalize_memo():
  """ Purpose: initalize memory to contain a set of blocks of d & v, randomly placed"""
  memory = [-TOTAL_UNITS]
  while True:
    block_size = max(1, int(random.normal(loc=d, scale=v)))
    if abs(memory[-1]) > block_size:
      memory.insert(-1, block_size)
      memory[-1] += block_size
    else:
      memory[-1] *= -1
      return memory

def is_full():
  """ Purpose: checks whether the memory is full. """
  # true only if (memory == TOTAL_UNITS)
  return all(block > 0 for block in memory)

def request():
  """ Purpose: generate mememory request. """
  # list comprehension*: for every block that is allocated (>0), store their index
  # enumerate(memory): loop through each elem in memory
  return max(1, int(random.normal(loc=d, scale=v)))

def release(memory):
  """ Purpose: randomly select an allocated block and free it. """
  # list comprehension*: for every block that is allocated (>0), store their index
  # enumerate(memory): loop through each elem in memory
  allocated = [i for i, block in enumerate(memory) if block > 0]
  # If the memory has no filled blocks, memory remains unchanged
  if not allocated:
    return memory
  # random.choice(): function from random library that will randomly select index of alloc. block and store
  idx = random.choice(allocated)
  # access index of the randomly selected block in memory; +(alloc.) will turn -(dealloc./empty/hole)
  memory[idx] *= -1
  # pass updated memory in coalesce to megre any adjacent free blocks
  return coalesce(memory)

def coalesce(memory):
  """ Purpose: merge adjacent holes. """
  # at the beginning of the memory
  i = 0
  # traverse every index in memory besides the last one (other we'll crash when checking i+1)
  while i < len(memory) - 1:
    # if the adjacent memory blocks are both holes (+alloc., -dealloc./empty/hole)
    if memory[i] < 0 and memory[i + 1] < 0:
      # add/merge them to create a larger hole (ex. memory[i]=-8, and memory[i+1]=-8, then -8+(-8)=-16)
      memory[i] += memory[i + 1]
      # deletes adj. block (now merged)
      del memory[i + 1]
    # if the not both holes, increment to the next index
    else:
      i += 1
  # update memory (may be shorter now if we've merged)
  return memory

def first_fit(s, memory):
   # Finds the index of the first hole large enough for size s.
    for i, block_size in enumerate(memory):
        if block_size < 0 and abs(block_size) >= s:
            # Found a suitable hole
            return i
    return -1 # No suitable hole found

def worst_fit(s, memory):
    holes = [(i, val) for i, val in enumerate(memory) if val < 0]
    if not holes:
      return -1
    return max(holes, key=lambda x: abs(x[1]))[0]
    # return min(holes, key=lambda x: x[1])[0]

def allocate(memory, i, s):
  """Purpose: allocate memory of size s, by splitting a hole."""
  # easier tracking of how much space is available in hole
  # ex. -20 => hole_size = 20, if we have an s=8, then remain. space 12
  hole_size = abs(memory[i])
  # removed the hole bc it will be edited (split and alloc. memory s)
  memory.pop(i)
  # replace hole with s
  memory.insert(i, s)
  # since we used abs, the hole_size will be a +int. Can check if there's space.
  # Ex. 12 > 8, so...
  if hole_size > s:
    # create a new block after allocating s.
    # Ex. (i + 1, -(20 - 8)) will insert a hole (-12) after the alloc. s (8) -> [... 8, -12...]
    memory.insert(i + 1, -(hole_size - s))
  # update memory (may be longer if there was space left after alloc.)
  return memory

def simulate(steps, strategy_fn):
    """ Purpose:  """
    # record data
    memory = initalize_memo()
    print(memory, sum(memory))

    utilization_record = []
    holes_examined_record = []

    for _ in range(steps):
        holes_examined = 0

        # Try to satisfy as many requests as possible
        while True:
            s = request()
            i = strategy_fn(s, memory)
            holes_examined += len([b for b in memory if b < 0])

            # If strategy found a hole
            if i != -1:
                memory = allocate(memory, i, s)
            else:
                break  # stop when request fails

        # Record utilization: sum of allocated blocks / TOTAL_UNITS
        used = sum(b for b in memory if b > 0)
        utilization = used / TOTAL_UNITS
        utilization_record.append(utilization)
        holes_examined_record.append(holes_examined)

        # Release a random block
        memory = release(memory)

    avg_util = sum(utilization_record) / len(utilization_record)
    avg_holes_examined = sum(holes_examined_record) / len(holes_examined_record)

    return avg_util, avg_holes_examined

## Varies Request Size (d)

In [ ]:
strategies = {
  "First-Fit": first_fit,
  "Worst-Fit": worst_fit,
}

d_values = list(range(5, 50, 5))  # Vary mean request size
steps = 100
results = {name: [] for name in strategies}

for d_val in d_values:
  d = d_val  # update global variable
  for name, strategy in strategies.items():
    avg_util, avg_holes = simulate(steps=steps, strategy_fn=strategy)
    results[name].append((avg_util, avg_holes))


[2, 6, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 6, 4, 4, 5, 3, 5, 4, 5, 4, 3] 100
[4, 5, 5, 4, 6, 6, 4, 5, 2, 5, 6, 4, 4, 5, 4, 3, 4, 3, 4, 5, 3, 4, 5] 100
[9, 11, 10, 10, 10, 10, 11, 8, 9, 10, 2] 100
[10, 10, 10, 10, 9, 9, 9, 9, 8, 10, 6] 100
[15, 16, 15, 15, 14, 13, 12] 100
[15, 15, 15, 17, 15, 14, 9] 100
[20, 20, 18, 20, 19, 3] 100
[18, 19, 20, 18, 20, 5] 100
[24, 24, 26, 25, 1] 100
[25, 24, 25, 23, 3] 100
[29, 30, 30, 11] 100
[29, 31, 28, 12] 100
[33, 36, 31] 100
[34, 34, 32] 100
[40, 40, 20] 100
[40, 40, 20] 100
[44, 44, 12] 100
[43, 45, 12] 100


## Compute Avg Memory Utilizations and Holes Examined (d)


In [ ]:
print("\nAverage Utilization and Holes Examined per Strategy:\n")
print(f"{'Request Size (d)':>15} | {'Strategy':>10} | {'Utilization':>12} | {'Holes Examined':>16}")
print("-" * 60)

for d_val in d_values:
  for name in strategies:
    util, holes = results[name][d_values.index(d_val)]
    print(f"{d_val:>15} | {name:>10} | {util:12.4f} | {holes:16.2f}")


Average Utilization and Holes Examined per Strategy:

Request Size (d) |   Strategy |  Utilization |   Holes Examined
------------------------------------------------------------
              5 |  First-Fit |       0.8876 |            13.08
              5 |  Worst-Fit |       1.8429 |             1.34
             10 |  First-Fit |       0.8560 |             8.17
             10 |  Worst-Fit |       2.8975 |             1.35
             15 |  First-Fit |       0.8376 |             4.98
             15 |  Worst-Fit |       3.6334 |             1.37
             20 |  First-Fit |       0.9003 |             3.62
             20 |  Worst-Fit |       3.9141 |             1.31
             25 |  First-Fit |       0.8634 |             3.40
             25 |  Worst-Fit |       5.0501 |             1.34
             30 |  First-Fit |       0.7961 |             3.17
             30 |  Worst-Fit |       6.7053 |             1.37
             35 |  First-Fit |       0.6754 |             2.63
 

In [ ]:
avg_util, avg_holes = simulate(UNIT_SIZE, first_fit)
print(f"One-off test run (First-Fit): Utilization = {avg_util:.2f}, Holes Examined = {avg_holes:.2f}")

avg_util, avg_holes = simulate(UNIT_SIZE, worst_fit)
print(f"One-off test run (Worst-Fit): Utilization = {avg_util:.2f}, Holes Examined = {avg_holes:.2f}")

[44, 44, 12] 100
One-off test run (First-Fit): Utilization = 0.91, Holes Examined = 1.62
[46, 46, 8] 100
One-off test run (Worst-Fit): Utilization = 1.71, Holes Examined = 1.25


## Varies Standard Deviation (v)


In [ ]:
strategies = {
  "First-Fit": first_fit,
  "Worst-Fit": worst_fit,
}

v_values = list(range(0, 5, 1))  # Vary standard deviation
steps = 100
results = {name: [] for name in strategies}

for v_val in v_values:
  v = v_val  # update global variable
  for name, strategy in strategies.items():
    avg_util, avg_holes = simulate(steps=steps, strategy_fn=strategy)
    results[name].append((avg_util, avg_holes))


[45, 45, 10] 100
[45, 45, 10] 100
[44, 44, 12] 100
[45, 42, 13] 100
[46, 41, 13] 100
[45, 45, 10] 100
[49, 48, 3] 100
[47, 53] 100
[43, 43, 14] 100
[43, 46, 11] 100


## Compute Avg Memory Utilizations and Holes Examined (v)


In [ ]:
print("\nAverage Utilization and Holes Examined per Strategy:\n")
print(f"{'Standard Deviation (v)':>25} | {'Strategy':>10} | {'Utilization':>12} | {'Holes Examined':>16}")
print("-" * 60)

for v_val in v_values:
  for name in strategies:
    util, holes = results[name][v_values.index(v_val)]
    print(f"{v_val:>25} | {name:>10} | {util:12.4f} | {holes:16.2f}")



Average Utilization and Holes Examined per Strategy:

   Standard Deviation (v) |   Strategy |  Utilization |   Holes Examined
------------------------------------------------------------
                        0 |  First-Fit |       0.9040 |             2.43
                        0 |  Worst-Fit |       1.3360 |             0.99
                        1 |  First-Fit |       0.8065 |             2.48
                        1 |  Worst-Fit |       8.7569 |             1.33
                        2 |  First-Fit |       0.8328 |             2.48
                        2 |  Worst-Fit |      11.0191 |             1.45
                        3 |  First-Fit |       0.8141 |             2.53
                        3 |  Worst-Fit |      10.8094 |             1.44
                        4 |  First-Fit |       0.8115 |             2.45
                        4 |  Worst-Fit |      11.5885 |             1.45


In [ ]:
avg_util, avg_holes = simulate(UNIT_SIZE, first_fit)
print(f"One-off test run (First-Fit): Utilization = {avg_util:.2f}, Holes Examined = {avg_holes:.2f}")

avg_util, avg_holes = simulate(UNIT_SIZE, worst_fit)
print(f"One-off test run (Worst-Fit): Utilization = {avg_util:.2f}, Holes Examined = {avg_holes:.2f}")

[44, 43, 13] 100
One-off test run (First-Fit): Utilization = 0.74, Holes Examined = 2.38
[43, 42, 15] 100
One-off test run (Worst-Fit): Utilization = 1.91, Holes Examined = 1.38


## Visualization

Heres the code when implementing the whole thing used to implement the full memory allocation simulator. It includes a popup-style GUI where you can:


-  Adjust memory size, request mean (d), and standard deviation (v)

Step through or auto-run the simulation

- Visualize how First-Fit and Worst-Fit handle allocation in real-time

- Track stats like utilization, holes examined, and failures

- Watch graphs update over time

The interface pops up when you run the file, and it shows two color-coded memory bars (green = used, red = free). You can play with the settings or just hit Start Auto to watch it in action.


- pip install pyqt5 matplotlib numpy

- Use VS code

In [ ]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
                            QPushButton, QLabel, QTabWidget, QGridLayout, QGroupBox,
                            QSpinBox, QDoubleSpinBox, QSplitter, QFrame)
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QPainter, QColor, QFont, QPen

# ----------------------
# SIMULATION FUNCTIONS
# ----------------------
def initialize_memory(total_memory, d=50, v=10):
    """Initialize memory with alternating allocated and free blocks."""
    memory = []
    used = 0
    while used < total_memory:
        block_size = max(1, int(random.gauss(d, v)))
        if used + block_size > total_memory:
            block_size = total_memory - used

        # 50% chance: allocated or hole
        seg = block_size if random.random() < 0.5 else -block_size
        memory.append(seg)
        used += block_size
    return memory

def first_fit(memory, request):
    """First-Fit strategy: allocate in first hole that fits."""
    holes_examined = 0
    for i, seg in enumerate(memory):
        if seg < 0:  # Found a free block (hole)
            holes_examined += 1
            available = abs(seg)
            if available >= request:
                remaining = available - request
                new_segments = [request]
                if remaining > 0:
                    new_segments.append(-remaining)
                new_memory = memory[:i] + new_segments + memory[i+1:]
                return True, holes_examined, new_memory
    return False, holes_examined, memory

def worst_fit(memory, request):
    """Worst-Fit strategy: allocate in largest available hole."""
    holes_examined = 0
    best_idx = -1
    best_free = -1
    for i, seg in enumerate(memory):
        if seg < 0:
            holes_examined += 1
            available = abs(seg)
            if available >= request and available > best_free:
                best_free = available
                best_idx = i
    if best_idx >= 0:
        available = abs(memory[best_idx])
        remaining = available - request
        new_segments = [request]
        if remaining > 0:
            new_segments.append(-remaining)
        new_memory = memory[:best_idx] + new_segments + memory[best_idx+1:]
        return True, holes_examined, new_memory
    return False, holes_examined, memory

def coalesce(memory):
    """Merge adjacent free blocks (holes) into one larger free block."""
    if not memory:
        return memory
    new_memory = []
    current = memory[0]
    for seg in memory[1:]:
        if current < 0 and seg < 0:
            current += seg  # Merge two free blocks (both negative)
        else:
            new_memory.append(current)
            current = seg
    new_memory.append(current)
    return new_memory

def release_random(memory):
    """Randomly release an allocated block and coalesce adjacent free blocks."""
    allocated_indices = [i for i, seg in enumerate(memory) if seg > 0]
    if not allocated_indices:
        return memory
    idx = random.choice(allocated_indices)
    memory[idx] = -memory[idx]
    return coalesce(memory)

def memory_utilization(memory):
    """Return memory utilization as fraction of allocated memory."""
    total = sum(abs(seg) for seg in memory)
    allocated = sum(seg for seg in memory if seg > 0)
    return allocated / total if total > 0 else 0

# ----------------------
# MEMORY VISUALIZATION WIDGET
# ----------------------
class MemoryView(QWidget):
    """Widget to visualize memory as segmented bar."""
    def __init__(self, memory=None, parent=None):
        super(MemoryView, self).__init__(parent)
        self.memory = memory if memory else []
        self.setMinimumHeight(40)

    def set_memory(self, memory):
        self.memory = memory
        self.update()

    def paintEvent(self, event):
        if not self.memory:
            return

        painter = QPainter(self)
        rect = self.rect()
        x = 0
        total_size = sum(abs(seg) for seg in self.memory)
        width_ratio = rect.width() / total_size
        height = rect.height()

        # Draw blocks
        for seg in self.memory:
            width = abs(seg) * width_ratio
            if seg > 0:  # Allocated block
                painter.setBrush(QColor(46, 204, 113))  # Green
                painter.setPen(QPen(QColor(39, 174, 96), 1))
            else:  # Free block (hole)
                painter.setBrush(QColor(231, 76, 60))  # Red
                painter.setPen(QPen(QColor(192, 57, 43), 1))

            painter.drawRect(int(x), 0, max(1, int(width)), height)

            # Draw size label if block is large enough
            if width > 20:
                painter.setPen(Qt.white)
                painter.setFont(QFont("Arial", 8))
                painter.drawText(int(x + 2), 2, int(width - 4), height - 4,
                                Qt.AlignCenter, str(abs(seg)))
            x += width

        # Draw utilization percentage at the top
        util = memory_utilization(self.memory) * 100
        painter.setPen(Qt.black)
        painter.setFont(QFont("Arial", 9, QFont.Bold))
        painter.drawText(rect, Qt.AlignTop | Qt.AlignRight, f"{util:.1f}%")

# ----------------------
# METRICS GRAPH WIDGET
# ----------------------
class MetricsGraph(FigureCanvas):
    """Widget to display performance metrics over time."""
    def __init__(self, parent=None):
        self.fig, self.axes = plt.subplots(1, 1, figsize=(5, 3), dpi=100)
        super(MetricsGraph, self).__init__(self.fig)
        self.setParent(parent)

        # History data
        self.utilization_history = {'First-Fit': [], 'Worst-Fit': []}
        self.search_history = {'First-Fit': [], 'Worst-Fit': []}
        self.steps = []

        # Setup
        self.axes.set_title('Memory Utilization', fontsize=10)
        self.axes.set_xlabel('Step', fontsize=8)
        self.axes.set_ylabel('Utilization %', fontsize=8)
        self.axes.tick_params(labelsize=8)
        self.fig.tight_layout()

    def update_metrics(self, step, first_fit_util, worst_fit_util,
                      first_fit_search, worst_fit_search):
        """Add new data points and redraw."""
        self.steps.append(step)
        self.utilization_history['First-Fit'].append(first_fit_util * 100)
        self.utilization_history['Worst-Fit'].append(worst_fit_util * 100)
        self.search_history['First-Fit'].append(first_fit_search)
        self.search_history['Worst-Fit'].append(worst_fit_search)

        self.plot_utilization()

    def plot_utilization(self):
        """Plot the utilization history."""
        self.axes.clear()
        self.axes.plot(self.steps, self.utilization_history['First-Fit'],
                     'b-', label='First-Fit')
        self.axes.plot(self.steps, self.utilization_history['Worst-Fit'],
                     'r-', label='Worst-Fit')
        self.axes.set_title('Memory Utilization', fontsize=10)
        self.axes.set_xlabel('Step', fontsize=8)
        self.axes.set_ylabel('Utilization %', fontsize=8)
        self.axes.tick_params(labelsize=8)
        self.axes.legend(fontsize=8)
        self.axes.grid(True, linestyle='--', alpha=0.7)
        self.fig.tight_layout()
        self.draw()

    def plot_search_time(self):
        """Plot the search time history."""
        self.axes.clear()
        self.axes.plot(self.steps, self.search_history['First-Fit'],
                     'b-', label='First-Fit')
        self.axes.plot(self.steps, self.search_history['Worst-Fit'],
                     'r-', label='Worst-Fit')
        self.axes.set_title('Avg Holes Examined', fontsize=10)
        self.axes.set_xlabel('Step', fontsize=8)
        self.axes.set_ylabel('Holes', fontsize=8)
        self.axes.tick_params(labelsize=8)
        self.axes.legend(fontsize=8)
        self.axes.grid(True, linestyle='--', alpha=0.7)
        self.fig.tight_layout()
        self.draw()

    def clear_history(self):
        """Reset all history data."""
        self.utilization_history = {'First-Fit': [], 'Worst-Fit': []}
        self.search_history = {'First-Fit': [], 'Worst-Fit': []}
        self.steps = []
        self.axes.clear()
        self.axes.set_title('Memory Utilization', fontsize=10)
        self.axes.set_xlabel('Step', fontsize=8)
        self.axes.set_ylabel('Utilization %', fontsize=8)
        self.draw()

# ----------------------
# MAIN SIMULATOR INTERFACE
# ----------------------
class MemorySimulator(QMainWindow):
    """Main application window for memory allocation simulator."""
    def __init__(self):
        super(MemorySimulator, self).__init__()
        self.setWindowTitle("Memory Allocation Strategy Simulator")
        self.setMinimumSize(1000, 600)

        # Simulation parameters
        self.memory_size = 1000
        self.request_mean = 50
        self.request_std = 20
        self.sim_speed = 500  # ms between steps

        # Simulation state
        self.step_count = 0
        self.first_fit_memory = initialize_memory(self.memory_size, self.request_mean, self.request_std)
        self.worst_fit_memory = self.first_fit_memory.copy()
        self.first_fit_stats = {'allocated': 0, 'failed': 0, 'holes_examined': 0}
        self.worst_fit_stats = {'allocated': 0, 'failed': 0, 'holes_examined': 0}

        # Setup UI
        self.setup_ui()

        # Timer for auto-stepping
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.step_simulation)

    def setup_ui(self):
        """Create the main UI layout."""
        central_widget = QWidget()
        main_layout = QVBoxLayout(central_widget)

        # Top controls
        controls_layout = QHBoxLayout()

        # Parameters group
        params_group = QGroupBox("Simulation Parameters")
        params_layout = QGridLayout(params_group)

        # Memory size
        params_layout.addWidget(QLabel("Memory Size:"), 0, 0)
        self.memory_size_spin = QSpinBox()
        self.memory_size_spin.setRange(100, 10000)
        self.memory_size_spin.setValue(self.memory_size)
        params_layout.addWidget(self.memory_size_spin, 0, 1)

        # Request mean
        params_layout.addWidget(QLabel("Request Mean (d):"), 1, 0)
        self.req_mean_spin = QSpinBox()
        self.req_mean_spin.setRange(1, 500)
        self.req_mean_spin.setValue(self.request_mean)
        params_layout.addWidget(self.req_mean_spin, 1, 1)

        # Request std dev
        params_layout.addWidget(QLabel("Request StdDev (v):"), 2, 0)
        self.req_std_spin = QSpinBox()
        self.req_std_spin.setRange(1, 200)
        self.req_std_spin.setValue(self.request_std)
        params_layout.addWidget(self.req_std_spin, 2, 1)

        # Simulation speed
        params_layout.addWidget(QLabel("Sim Speed (ms):"), 3, 0)
        self.speed_spin = QSpinBox()
        self.speed_spin.setRange(10, 2000)
        self.speed_spin.setValue(self.sim_speed)
        params_layout.addWidget(self.speed_spin, 3, 1)

        controls_layout.addWidget(params_group)

        # Action buttons group
        buttons_group = QGroupBox("Controls")
        buttons_layout = QGridLayout(buttons_group)

        # Initialize button
        self.init_button = QPushButton("Initialize")
        self.init_button.clicked.connect(self.initialize_simulation)
        buttons_layout.addWidget(self.init_button, 0, 0)

        # Step button
        self.step_button = QPushButton("Step")
        self.step_button.clicked.connect(self.step_simulation)
        buttons_layout.addWidget(self.step_button, 0, 1)

        # Start/Stop button
        self.run_button = QPushButton("Start Auto")
        self.run_button.clicked.connect(self.toggle_auto_run)
        buttons_layout.addWidget(self.run_button, 1, 0)

        # Reset button
        self.reset_button = QPushButton("Reset")
        self.reset_button.clicked.connect(self.reset_simulation)
        buttons_layout.addWidget(self.reset_button, 1, 1)

        controls_layout.addWidget(buttons_group)

        # Stats group
        stats_group = QGroupBox("Statistics")
        stats_layout = QGridLayout(stats_group)

        # First-Fit stats
        stats_layout.addWidget(QLabel("First-Fit:"), 0, 0)
        self.ff_stats_label = QLabel("Allocated: 0, Failed: 0, Avg Holes: 0.0")
        stats_layout.addWidget(self.ff_stats_label, 0, 1)

        # Worst-Fit stats
        stats_layout.addWidget(QLabel("Worst-Fit:"), 1, 0)
        self.wf_stats_label = QLabel("Allocated: 0, Failed: 0, Avg Holes: 0.0")
        stats_layout.addWidget(self.wf_stats_label, 1, 1)

        # Current step
        stats_layout.addWidget(QLabel("Current Step:"), 2, 0)
        self.step_label = QLabel("0")
        stats_layout.addWidget(QLabel("Current Request:"), 3, 0)
        self.current_request_label = QLabel("—")
        stats_layout.addWidget(self.current_request_label, 3, 1)
        stats_layout.addWidget(self.step_label, 2, 1)

        controls_layout.addWidget(stats_group)
        main_layout.addLayout(controls_layout)

        # Memory view section
        memory_group = QGroupBox("Memory Visualization")
        memory_layout = QVBoxLayout(memory_group)

        # First-Fit memory view
        ff_layout = QVBoxLayout()
        ff_layout.addWidget(QLabel("First-Fit Memory:"))
        self.ff_view = MemoryView(self.first_fit_memory)
        ff_layout.addWidget(self.ff_view)
        memory_layout.addLayout(ff_layout)

        # Worst-Fit memory view
        wf_layout = QVBoxLayout()
        wf_layout.addWidget(QLabel("Worst-Fit Memory:"))
        self.wf_view = MemoryView(self.worst_fit_memory)
        wf_layout.addWidget(self.wf_view)
        memory_layout.addLayout(wf_layout)

        main_layout.addWidget(memory_group)

        # Metrics visualization
        metrics_group = QGroupBox("Performance Metrics")
        metrics_layout = QVBoxLayout(metrics_group)

        # Graph selection buttons
        graph_buttons = QHBoxLayout()

        self.util_button = QPushButton("Show Utilization")
        self.util_button.clicked.connect(lambda: self.metrics_graph.plot_utilization())
        graph_buttons.addWidget(self.util_button)

        self.search_button = QPushButton("Show Search Time")
        self.search_button.clicked.connect(lambda: self.metrics_graph.plot_search_time())
        graph_buttons.addWidget(self.search_button)

        metrics_layout.addLayout(graph_buttons)

        # Metrics graph
        self.metrics_graph = MetricsGraph()
        metrics_layout.addWidget(self.metrics_graph)

        main_layout.addWidget(metrics_group)

        self.setCentralWidget(central_widget)

    def initialize_simulation(self):
        """Initialize simulation with current parameters."""
        # Get values from UI
        self.memory_size = self.memory_size_spin.value()
        self.request_mean = self.req_mean_spin.value()
        self.request_std = self.req_std_spin.value()
        self.sim_speed = self.speed_spin.value()

        # Reset simulation state
        self.reset_simulation()

    def reset_simulation(self):
        """Reset the simulation to initial state."""
        # Stop auto-run if active
        if self.timer.isActive():
            self.toggle_auto_run()

        # Reset state
        self.step_count = 0
        self.first_fit_memory = initialize_memory(self.memory_size, self.request_mean, self.request_std)
        self.worst_fit_memory = self.first_fit_memory.copy()
        self.first_fit_stats = {'allocated': 0, 'failed': 0, 'holes_examined': 0}
        self.worst_fit_stats = {'allocated': 0, 'failed': 0, 'holes_examined': 0}

        # Update UI
        self.ff_view.set_memory(self.first_fit_memory)
        self.wf_view.set_memory(self.worst_fit_memory)
        self.step_label.setText(str(self.step_count))
        self.update_stats_display()

        # Clear metrics history
        self.metrics_graph.clear_history()

    def step_simulation(self):
        """Execute one step of the simulation."""
        # Generate a memory request
        request_size = max(1, min(self.memory_size - 1,
                                int(random.gauss(self.request_mean, self.request_std))))

        # update the “current request” label
        self.current_request_label.setText(f"{request_size}")

        # Apply First-Fit
        ff_success, ff_holes, ff_new_memory = first_fit(self.first_fit_memory, request_size)
        if ff_success:
            self.first_fit_memory = ff_new_memory
            self.first_fit_stats['allocated'] += 1
            self.first_fit_stats['holes_examined'] += ff_holes
        else:
            self.first_fit_stats['failed'] += 1
            # Release random block on failure
            self.first_fit_memory = release_random(self.first_fit_memory)

        # Apply Worst-Fit
        wf_success, wf_holes, wf_new_memory = worst_fit(self.worst_fit_memory, request_size)
        if wf_success:
            self.worst_fit_memory = wf_new_memory
            self.worst_fit_stats['allocated'] += 1
            self.worst_fit_stats['holes_examined'] += wf_holes
        else:
            self.worst_fit_stats['failed'] += 1
            # Release random block on failure
            self.worst_fit_memory = release_random(self.worst_fit_memory)

        # Update step count
        self.step_count += 1

        # Update UI
        self.ff_view.set_memory(self.first_fit_memory)
        self.wf_view.set_memory(self.worst_fit_memory)
        self.step_label.setText(str(self.step_count))
        self.update_stats_display()

        # Calculate metrics for graphs
        ff_util = memory_utilization(self.first_fit_memory)
        wf_util = memory_utilization(self.worst_fit_memory)

        ff_avg_search = 0
        if self.first_fit_stats['allocated'] > 0:
            ff_avg_search = self.first_fit_stats['holes_examined'] / self.first_fit_stats['allocated']

        wf_avg_search = 0
        if self.worst_fit_stats['allocated'] > 0:
            wf_avg_search = self.worst_fit_stats['holes_examined'] / self.worst_fit_stats['allocated']

        # Update metrics graph
        self.metrics_graph.update_metrics(self.step_count, ff_util, wf_util,
                                         ff_avg_search, wf_avg_search)

    def toggle_auto_run(self):
        """Start or stop automatic simulation stepping."""
        if self.timer.isActive():
            self.timer.stop()
            self.run_button.setText("Start Auto")
        else:
            self.timer.start(self.sim_speed)  # Start timer with current speed
            self.run_button.setText("Stop Auto")

    def update_stats_display(self):
        """Update statistics display labels."""
        # First-Fit stats
        ff_allocated = self.first_fit_stats['allocated']
        ff_failed = self.first_fit_stats['failed']
        ff_avg_holes = 0
        if ff_allocated > 0:
            ff_avg_holes = self.first_fit_stats['holes_examined'] / ff_allocated

        self.ff_stats_label.setText(
            f"Allocated: {ff_allocated}, Failed: {ff_failed}, Avg Holes: {ff_avg_holes:.2f}")

        # Worst-Fit stats
        wf_allocated = self.worst_fit_stats['allocated']
        wf_failed = self.worst_fit_stats['failed']
        wf_avg_holes = 0
        if wf_allocated > 0:
            wf_avg_holes = self.worst_fit_stats['holes_examined'] / wf_allocated

        self.wf_stats_label.setText(
            f"Allocated: {wf_allocated}, Failed: {wf_failed}, Avg Holes: {wf_avg_holes:.2f}")

# ----------------------
# MAIN EXECUTION
# ----------------------
def main():
    app = QApplication(sys.argv)
    window = MemorySimulator()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()